In [7]:
import sys
import pathlib
import torch

import numpy as np

# Local
cwd = pathlib.Path().resolve()
src = cwd.parent
root = src.parent
sys.path.append(str(src))

from utils.watertopo import WaterTopo
from models.unet import UNet
from utils.animation import compare_animations
from utils.utils import recursive_pred

# Important for animation to run inline
%matplotlib inline
%matplotlib ipympl

In [8]:
#initialize GPU -  In case of windows use cuda instead of nps
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Is CUDA enabled?",torch.cuda.is_available())
print("Number of GPUs",torch.cuda.device_count())
print('Using device:', device)

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Is CUDA enabled? False
Number of GPUs 0
Using device: cpu


In [9]:
grid_size = 64

In [12]:
from utils.animation import compare_animations

save_path = "../results/trained_models/unet/"
models = [UNet(2, [32,64], 1).load_state_dict(
    torch.load(save_path + "unet_32_64_orig_data80_skip0", 
               map_location="cpu")),

               UNet(2, [32,64], 1).load_state_dict(
    torch.load(save_path + "unet_32_64_orig_data80_skip5", 
               map_location="cpu")),

               UNet(2, [32,64], 1).load_state_dict(
    torch.load(save_path + "unet_32_64_orig_data80_skip10", 
               map_location="cpu"))]

skips = [0, 5, 10]
labels = ["0-skip", "5-skip", "10-skip"]

sim = WaterTopo.load_simulations(str(root)+"/data/normalized_data/test1", 
                                 1, grid_size, True)[0]

topo = sim.topography.reshape([1, grid_size, grid_size])
wd_0 = sim.wd[0].reshape([1, grid_size, grid_size])
X = np.concatenate([topo, wd_0])

wds = []
wds.append(sim.wd)

with torch.no_grad():
    for i in range(len(models)):
        model = models[i]
        model.eval()
        skip = skips[i]

        if skips:
            sim_skips = sim.implement_skips(skip)

        wd = sim_skips.wd

        outputs = recursive_pred(model, X, sim_skips.wd.shape[0]-1, include_first_timestep=True)
        wds.append(outputs)

    compare_animations(wds, labels)

AttributeError: '_IncompatibleKeys' object has no attribute 'eval'